# MGSC 404 Foundations of Decision Analytics 


## Team Members: Ali Dunn, James Martignetti, Nicholas Tariro Toronga, Sean Mitro

In [217]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import pprint
import numpy as np
import datetime
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import datascience
import folium 
from folium import plugins

import requests
from dateutil.parser import parse

from gurobipy import *
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import math

**Assumptions**

Our model relies on the following assumptions:

1. Capacity of each zone has to be between 20%-80%
2. Bike rebalancing is carried out at the beginning of each time period. So for example, everyday at 12:00am (midnight), 6:00am, 12:00pm (noon), and 6:00pm, for a total of four daily rebalances.
3. The system will be static while rebalancing happens - meaning that people cannot take out or park bikes during this time.
4. Customer satisfaction will be higher/lower during the rush hour periods should a bike be available/unavailable. 
5. Assume that there is no difference between unavailability of bikes and unavailability of docks with regards to customer satisfaction.
6. Assume that there is no difference between availability of bikes and availability of docks with regards to customer satisfaction.
7. The bikes operating during the shift will be fixed - i.e. no broken bikes, no stolen bikes. ADD NUMBER OF BIKES IN THE SYSTEM FOR GREATER ACCURACY!?

POSSIBLE ASSUMPTION ONCE BASICS RUN PROPERLY: Cost of moving one bike from i to j (Cij) is more expensive during the night shift

**Sets**

Let z represesnt the number of zones

$Z = ${0, 1, 2,..., N-1} 

where N = 20 representing the twenty different clusters that we have identified. 

Let T represent time

$T = ${0, 1, 2, 3} 

where t = 0 represents the time period between 12:00am (midnight) and 5:59am, t = 1 represents the time period between 6:00am and 11.59am, t = 2 represents the time period between 12:00pm (noon) and 6:00pm, and t = 3 represents the time period between 6:00pm and 11:59pm. We identify time periods t = 1 (6:00am to 11:59am) and t = 2 (12:00pm (noon) to 6:00pm) as the rush hour periods based on the revelations of our visualizations.

**Decision Variable**


We define the following decision variable where i = 1, 2, 3,..., 20  (Number of zones (departure)), j = 1, 2, 3,..., 20 (Number of zones (arrival)) and t = 1,2,3,4 (time periods):

$i\in Z$

$j\in Z$

$t\in T$


1. $X_{ijt}$ = Number of bikes to move from $i\in Z$ to $j\in Z$ at time $t\in T$


**State Variables**

$i\in Z$

$t\in T$

We define the following state variable where $i=1, 2, 3,..., 20$ (Number of zones), and t = 1,2,3,4 (time periods): 

1. $I_{it}$ = Number of bikes at zone $i\in I$ at time $t\in T$

**Parameters**

Data analysis reveals the following parameters:

1. $I^M_i$ = Capacity for $i \in I$ (each zone) 

    - $I^M_0$ = 352
    - $I^M_1$ = 869
    - $I^M_2$ = 702
    - $I^M_3$ = 195
    - $I^M_4$ = 738
    - $I^M_5$ = 407
    - $I^M_6$ = 249
    - $I^M_7$ = 236
    - $I^M_8$ = 147
    - $I^M_9$ = 402
    - $I^M_{10}$ = 248
    - $I^M_{11}$ = 240
    - $I^M_{12}$ = 792
    - $I^M_{13}$ = 2067
    - $I^M_{14}$ = 426
    - $I^M_{15}$ = 472
    - $I^M_{16}$ = 768
    - $I^M_{17}$ = 157
    - $I^M_{18}$ = 718
    - $I^M_{19}$ = 205
    
    
2. $C_{ijt}$ = Change in customer satisfaction resulting from moving one bike from i to j during time t.

where $C_{ijt}$ can take four possible values: -4, -2, 2, and 4. These "scoring" values are contingent upon the time period. Scores on the extreme ends (-4 and 4) are granted during rush hour periods (time periods t = 1 and t = 2: $C_{ij1}$ and $C_{ij2}$). Scores of -2 and 2 are given during non-rush hour periods (i.e. time periods t = 0 and t = 3: $C_{ij0}$ and $C_{ij3}$).

3. $P_{ijt}$ = Number of trips from i to j
4. $P_{jit}$ = Number of trips from j to i

**Objective Function**

$$\max\sum_{i\in Z}\sum_{j\in Z}\sum_{t\in T}\bigg(X_{ijt} * C_{ijt}\bigg)$$

**Constraints**

***Capacity Constraint - Demand:***

Zone capacity must be between 20%-80% at all times so that we are not not underpopulating the stations in the zone during higher rush hours, and not overpopulating them during hours of lesser ridership. 


$$0.20 * I^M_i \leq I_{it} \leq 0.80 * I^M_i$$

$$\forall i\in Z$$


$$\forall t\in T$$



***Capacity Constraints - Docks:***

The numbers of bikes moved from zone j to zone i cannot exceed the current non-availability of bikes in zone i at time t (which is the difference between the capacity of zone i and the number of bikes at zone i in time t). We do not want lack of parking space for the bikes to be an issue due to rebalancing.


$$I^M_i - I_{it} \geq X_{jit}$$

$$\forall i\in Z$$


$$\forall j\in Z$$


$$\forall t\in T$$





***System Dynamics Constraint:***

The system dynamics constraint ensures a stable amount of bicycles at each zone at all times. The number of bikes at zone i at time t must be equal to the number of bikes at zone i at time t-1 plus the difference between bikes moved from zone j to i and bikes moved from zone i to j due to rebalancing. The difference in the number of bikes not caused by rebalancing, but rather caused by ridership must also be accounted for by subtracting the difference between outgoing trips (zone j to zone i) and incoming trips (zone i to zone j). 

$$I_{it} = I_{i(t-1)} + \sum_{j\in Z}X_{jit} - \sum_{j\in Z}X_{ijt} + \sum_{j\in Z}P_{jit} - \sum_{j\in Z}P_{ijt}$$


$$\forall i\in Z$$


$$\forall t\in T$$


***Non-Negative Variables:***

Zone capacity and bike movement cannot be negative.

$$I^M_i \geq 0$$

$$X_{ij} \geq 0$$

In [264]:
X = {} #Number of bikes to move from i E Z to j E Z at time t E T
I = {} #Number of bikes at zone i E I at time t E T
C = {} #Customer satisfaction
IJ = {}
JI = {}
r = {}
s={}

#creating model and adding variables to it
model = Model('Divvy')
for i in range(20): #N-1 = number of zones
    for j in range(20): 
        for t in range(4): #possibly switch t to be first loop 
            X[i,j,t] = model.addVar(lb=0)
            C[i,j,t] = model.addVar(lb=0)

for i in range(20):
    for t in range(4): #maybe a cleaner way to do this by combining above? 
            I[i,t] = model.addVar (lb=0)
            #JI[i,t] = model.addVar (lb=0)
            #IJ[i,t] = model.addVar (lb=0)

for i in range(20):
    for t in range(4):
         s[i,t] = model.addVar() #lb=0
    

for i in range(20):
    for t in range(4):
        r[i,t] = model.addVar()
            

In [265]:
#Parameters

IM = [352,869,702,195,738,407,249,236,147,402,248,240,792,2067,426,472,768,157,718,205] #Capacity per zone, we are this as base for t-1
IJ = [[0,23,7,1,10,4,0,0,0,1,1,1,7,32,0,6,16,0,11,0], [8,148,60,2,61,11,0,4,0,6,1,5,60,250,7,14,153,0,51,0], [29,306,198,18,105,38,0,7,5,21,3,4,180,749,17,55,435,1,124,1]]
JI = [[0,19,13,1,11,5,0,0,0,3,0,1,11,23,2,4,14,0,13,0], [11,124,53,2,51,14,0,3,0,6,2,2,51,298,5,15,159,0,45,0], [42,331,176,18,96,49,0,5,2,14,2,4,171,776,19,45,433,1,112,0]]

B = 6000 #total number of bikes
N = 20 #total number of zones
T = 4 #time periods
IJ[1][2]
#adding a variable threshold for the acceptable level of inventory for a stable system
var = 0.5
min_var = 0.2
max_var = 0.8

In [266]:
#creating capacity constraint
con_capacity_demand = {}
for t in range(4):
    for i in range(20):
         con_capacity_demand[i,t] = model.addConstr( I[i,t] <= IM[i])

In [267]:
#creating zone capacity constraint
con_capacity_docks = {}
for t in range(4):
    for i in range(20):
        for j in range(20):
              con_capacity_docks[i,j,t] = model.addConstr(IM[i] - I[i,t] >= X[i,j,t]) 

In [268]:
#constraint for the lower limit
con_sat1={}
for i in range(20):
    for t in range(4):
        con_sat[i,t] = model.addConstr( s[i,t] >= min_var - I[i,t]/IM[i])
        
    

In [269]:
#The other constraint for the upper limit
con_sat2 = {}
for i in range(20):
    for t in range(4):
        con_sat2[i,t] = model.addConstr( r[i,t] >= I[i,t]/IM[i]-max_var)


In [270]:
#creating system dynamics constraint


con_dynamics = {}   


for i in range(20):
    for t in range(4):
        if (t==0): 
            con_dynamics[i,t] = model.addConstr(I[i,t] == var*IM[i])
        else: 
            con_dynamics[i,t] = model.addConstr(I[i,t] == I[i,t-1] - quicksum(X[j,i,t] for j in np.arange(20)) - quicksum(X[i,j,t] for j in range(20))) # + IJ[t-1][i] - JI[t-1][i])

In [283]:
N = 20
T = 4

obj = 0
for i in range(20):
    for j in range(20):
        for t in range(4): 
            obj = obj + quicksum( quicksum((I[i,t]*5) for t in range(4)) for i in range(20)) +  5*( (quicksum( quicksum((s[i,t]) for t in range(4)) for i in range(20))) + (quicksum( quicksum((s[i,t]) for t in range(4)) for i in range(20))))
                                    
model.setObjective(obj, GRB.MINIMIZE)

model.optimize()

Optimize a model with 1920 rows, 3440 columns and 6080 nonzeros
Coefficient statistics:
  Matrix range     [5e-04, 2e+00]
  Objective range  [8e+03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1752000e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  4.175200000e+07


# Sensitivity Analysis

Suppose the threshold for system dynamics was increased to 0.55, how will this change affect the objective function

In [289]:
def dynamics_threshold_change(var):
    X = {} #Number of bikes to move from i E Z to j E Z at time t E T
    I = {} #Number of bikes at zone i E I at time t E T
    C = {} #Customer satisfaction
    IJ = {}
    JI = {}
    r = {}
    s={}

    #creating model and adding variables to it
    model = Model('Divvy')
    for i in range(20): #N-1 = number of zones
        for j in range(20): 
            for t in range(4): #possibly switch t to be first loop 
                X[i,j,t] = model.addVar(lb=0)
                C[i,j,t] = model.addVar(lb=0)

    for i in range(20):
        for t in range(4): #maybe a cleaner way to do this by combining above? 
                I[i,t] = model.addVar (lb=0)
                #JI[i,t] = model.addVar (lb=0)
                #IJ[i,t] = model.addVar (lb=0)

    for i in range(20):
        for t in range(4):
             s[i,t] = model.addVar() #lb=0


    for i in range(20):
        for t in range(4):
            r[i,t] = model.addVar()


    #Parameters

    IM = [352,869,702,195,738,407,249,236,147,402,248,240,792,2067,426,472,768,157,718,205] #Capacity per zone, we are this as base for t-1
    IJ = [[0,23,7,1,10,4,0,0,0,1,1,1,7,32,0,6,16,0,11,0], [8,148,60,2,61,11,0,4,0,6,1,5,60,250,7,14,153,0,51,0], [29,306,198,18,105,38,0,7,5,21,3,4,180,749,17,55,435,1,124,1]]
    JI = [[0,19,13,1,11,5,0,0,0,3,0,1,11,23,2,4,14,0,13,0], [11,124,53,2,51,14,0,3,0,6,2,2,51,298,5,15,159,0,45,0], [42,331,176,18,96,49,0,5,2,14,2,4,171,776,19,45,433,1,112,0]]

    B = 6000 #total number of bikes
    N = 20 #total number of zones
    T = 4 #time periods
    IJ[1][2]
    #adding a variable threshold for the acceptable level of inventory for a stable system
    

    #creating capacity constraint
    con_capacity_demand = {}
    for t in range(4):
        for i in range(20):
             con_capacity_demand[i,t] = model.addConstr( I[i,t] <= IM[i])

    #creating zone capacity constraint
    con_capacity_docks = {}
    for t in range(4):
        for i in range(20):
            for j in range(20):
                  con_capacity_docks[i,j,t] = model.addConstr(IM[i] - I[i,t] >= X[i,j,t]) 

    #constraint for the lower limit
    con_sat1={}
    for i in range(20):
        for t in range(4):
            con_sat[i,t] = model.addConstr( s[i,t] >= min_var - I[i,t]/IM[i])



    #The other constraint for the upper limit
    con_sat2 = {}
    for i in range(20):
        for t in range(4):
            con_sat2[i,t] = model.addConstr( r[i,t] >= I[i,t]/IM[i]-max_var)




    #creating system dynamics constraint


    con_dynamics = {}   


    for i in range(20):
        for t in range(4):
            if (t==0): 
                con_dynamics[i,t] = model.addConstr(I[i,t] == var*IM[i])
            else: 
                con_dynamics[i,t] = model.addConstr(I[i,t] == I[i,t-1] - quicksum(X[j,i,t] for j in np.arange(20)) - quicksum(X[i,j,t] for j in range(20))) # + IJ[t-1][i] - JI[t-1][i])

    N = 20
    T = 4

    obj = 0
    for i in range(20):
        for j in range(20):
            for t in range(4): 
                obj = obj + quicksum( quicksum((I[i,t]*5) for t in range(4)) for i in range(20)) +  5*( (quicksum( quicksum((s[i,t]) for t in range(4)) for i in range(20))) + (quicksum( quicksum((s[i,t]) for t in range(4)) for i in range(20))))

        model.setObjective(obj, GRB.MINIMIZE)

        model.optimize()
        return model.objVal


In [290]:
dynamics_threshold_change(0.4)

Optimize a model with 1920 rows, 3440 columns and 6080 nonzeros
Coefficient statistics:
  Matrix range     [5e-04, 2e+00]
  Objective range  [4e+02, 8e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 2e+03]
Presolve removed 1920 rows and 3440 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6720000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  1.672000000e+06


1672000.0

Changing the lower and upper bounds that define satisfaction

In [291]:
def satisfaction_bounds(min_var1, max_var1):
    X = {} #Number of bikes to move from i E Z to j E Z at time t E T
    I = {} #Number of bikes at zone i E I at time t E T
    C = {} #Customer satisfaction
    IJ = {}
    JI = {}
    r = {}
    s={}

    #creating model and adding variables to it
    model = Model('Divvy')
    for i in range(20): #N-1 = number of zones
        for j in range(20): 
            for t in range(4): #possibly switch t to be first loop 
                X[i,j,t] = model.addVar(lb=0)
                C[i,j,t] = model.addVar(lb=0)

    for i in range(20):
        for t in range(4): #maybe a cleaner way to do this by combining above? 
                I[i,t] = model.addVar (lb=0)
                #JI[i,t] = model.addVar (lb=0)
                #IJ[i,t] = model.addVar (lb=0)

    for i in range(20):
        for t in range(4):
             s[i,t] = model.addVar() #lb=0


    for i in range(20):
        for t in range(4):
            r[i,t] = model.addVar()


    #Parameters

    IM = [352,869,702,195,738,407,249,236,147,402,248,240,792,2067,426,472,768,157,718,205] #Capacity per zone, we are this as base for t-1
    IJ = [[0,23,7,1,10,4,0,0,0,1,1,1,7,32,0,6,16,0,11,0], [8,148,60,2,61,11,0,4,0,6,1,5,60,250,7,14,153,0,51,0], [29,306,198,18,105,38,0,7,5,21,3,4,180,749,17,55,435,1,124,1]]
    JI = [[0,19,13,1,11,5,0,0,0,3,0,1,11,23,2,4,14,0,13,0], [11,124,53,2,51,14,0,3,0,6,2,2,51,298,5,15,159,0,45,0], [42,331,176,18,96,49,0,5,2,14,2,4,171,776,19,45,433,1,112,0]]

    B = 6000 #total number of bikes
    N = 20 #total number of zones
    T = 4 #time periods
    IJ[1][2]
    #adding a variable threshold for the acceptable level of inventory for a stable system
    

    #creating capacity constraint
    con_capacity_demand = {}
    for t in range(4):
        for i in range(20):
             con_capacity_demand[i,t] = model.addConstr( I[i,t] <= IM[i])

    #creating zone capacity constraint
    con_capacity_docks = {}
    for t in range(4):
        for i in range(20):
            for j in range(20):
                  con_capacity_docks[i,j,t] = model.addConstr(IM[i] - I[i,t] >= X[i,j,t]) 

    #constraint for the lower limit
    con_sat1={}
    for i in range(20):
        for t in range(4):
            con_sat[i,t] = model.addConstr( s[i,t] >= min_var - I[i,t]/IM[i])



    #The other constraint for the upper limit
    con_sat2 = {}
    for i in range(20):
        for t in range(4):
            con_sat2[i,t] = model.addConstr( r[i,t] >= I[i,t]/IM[i]-max_var)




    #creating system dynamics constraint


    con_dynamics = {}   


    for i in range(20):
        for t in range(4):
            if (t==0): 
                con_dynamics[i,t] = model.addConstr(I[i,t] == var*IM[i])
            else: 
                con_dynamics[i,t] = model.addConstr(I[i,t] == I[i,t-1] - quicksum(X[j,i,t] for j in np.arange(20)) - quicksum(X[i,j,t] for j in range(20))) # + IJ[t-1][i] - JI[t-1][i])

    N = 20
    T = 4

    obj = 0
    for i in range(20):
        for j in range(20):
            for t in range(4): 
                obj = obj + quicksum( quicksum((I[i,t]*5) for t in range(4)) for i in range(20)) +  5*( (quicksum( quicksum((s[i,t]) for t in range(4)) for i in range(20))) + (quicksum( quicksum((s[i,t]) for t in range(4)) for i in range(20))))

        model.setObjective(obj, GRB.MINIMIZE)

        model.optimize()
        return model.objVal

    

In [294]:
satisfaction_bounds(0.2, 0.6)

Optimize a model with 1920 rows, 3440 columns and 6080 nonzeros
Coefficient statistics:
  Matrix range     [5e-04, 2e+00]
  Objective range  [4e+02, 8e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 2e+03]
Presolve removed 1920 rows and 3440 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0876000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  2.087600000e+06


2087600.0